<a href="https://colab.research.google.com/github/tvpian/Kafka-Stream-Processing/blob/main/docs%20/tutorials/Kafka_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rodrigo-arenas/kafkaml-anomaly-detection.git

Cloning into 'kafkaml-anomaly-detection'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 175 (delta 88), reused 134 (delta 55), pack-reused 0
Receiving objects: 100% (175/175), 9.91 MiB | 22.35 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [2]:
cd /content/kafkaml-anomaly-detection

/content/kafkaml-anomaly-detection


In [3]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.9 MB/s eta 0:00:00


In [4]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer, KafkaConsumer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd

ModuleNotFoundError: No module named 'kafka'

In [17]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.5.2/kafka_2.13-3.5.2.tgz
!tar -xzf kafka_2.13-3.5.2.tgz

In [18]:
!./kafka_2.13-3.5.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.5.2/config/zookeeper.properties
!./kafka_2.13-3.5.2/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.5.2/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10


Waiting for 10 secs until kafka and zookeeper services are up and running


In [19]:
!ps -ef | grep kafka


root        8526     659  0 05:24 ?        00:00:00 /bin/bash -c ps -ef | grep kafka
root        8528    8526  0 05:24 ?        00:00:00 grep kafka


In [23]:
# !./kafka_2.13-3.5.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic susy-train
# !./kafka_2.13-3.5.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic susy-test

Created topic susy-train.
Created topic susy-test.


In [24]:
!./kafka_2.13-3.5.2/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --topic transactions --create --partitions 3 --replication-factor 1

Created topic transactions.


In [26]:
!./kafka_2.13-3.5.2/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --topic anomalies --create --partitions 3 --replication-factor 1

Created topic anomalies.


In [25]:
from joblib import dump
import numpy as np
from sklearn.ensemble import IsolationForest

rng = np.random.RandomState(42)
# Generate train data
X = 0.3 * rng.randn(500, 2)
X_train = np.r_[X + 2, X - 2]
X_train = np.round(X_train, 3)

# fit the model
clf = IsolationForest(n_estimators=50, max_samples=500, random_state=rng, contamination=0.01)
clf.fit(X_train)

dump(clf, './isolation_forest.joblib')

['./isolation_forest.joblib']

In [37]:

def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message, key in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

# write_to_kafka("susy-train", zip(x_train, y_train))
# write_to_kafka("susy-test", zip(x_test, y_test))

In [61]:
TRANSACTIONS_TOPIC = "transactions"
OUTLIERS_GENERATION_PROBABILITY = 0.7
DELAY = 0.01
TRANSACTIONS_CONSUMER_GROUP = "transactions"
NUM_PARTITIONS = 3
ANOMALIES_TOPIC = "anomalies"
ANOMALIES_CONSUMER_GROUP = "anomalies"

In [ ]:
import json
import random
import time
from datetime import datetime
import numpy as np


_id = 0
producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])

if producer is not None:
    while _id<1000:
        # Generate some abnormal observations
        if random.random() <= OUTLIERS_GENERATION_PROBABILITY:
            X_test = np.random.uniform(low=-4, high=4, size=(1, 2)).tolist()
        else:
            X = 0.3 * np.random.randn(1, 2)
            X_test = (X + np.random.choice(a=[2, -2], size=1, p=[0.5, 0.5]))
            X_test = np.round(X_test, 3).tolist()

        current_time = datetime.utcnow().isoformat()
        print(record)
        record = {"id": _id, "data": X_test, "current_time": current_time}
        record = json.dumps(record).encode("utf-8")

        producer.send(topic=TRANSACTIONS_TOPIC,
                         value=record,
                         key=str(_id).encode('utf-8')).add_errback(error_callback)
        producer.flush()
        _id += 1
        time.sleep(DELAY)

In [74]:
!./kafka_2.13-3.5.2/bin/kafka-console-consumer.sh --bootstrap-server 127.0.0.1:9092 --topic transactions  --from-beginning --max-messages 10

{"id": 4, "data": [[0.3144653901201675, 2.5618138956780223]], "current_time": "2024-04-27T05:53:07.529469"}
{"id": 6, "data": [[3.3512449865829463, 3.7689977665826184]], "current_time": "2024-04-27T05:53:07.554834"}
{"id": 10, "data": [[2.058, 2.012]], "current_time": "2024-04-27T05:53:07.604694"}
{"id": 12, "data": [[-2.305173790219178, -2.9625789012955517]], "current_time": "2024-04-27T05:53:07.630644"}
{"id": 13, "data": [[1.2079448395365233, -1.0658623090674242]], "current_time": "2024-04-27T05:53:07.642850"}
{"id": 14, "data": [[2.264618535639734, 3.104725499305074]], "current_time": "2024-04-27T05:53:07.655069"}
{"id": 18, "data": [[0.2997946606215729, 0.40751176649028586]], "current_time": "2024-04-27T05:53:07.704825"}
{"id": 19, "data": [[-2.0666015252476093, 0.19319531374695487]], "current_time": "2024-04-27T05:53:07.717326"}
{"id": 20, "data": [[-2.2373495236108365, -0.6640118717547212]], "current_time": "2024-04-27T05:53:07.729931"}
{"id": 24, "data": [[2.772489206305634, -1

In [81]:
import json
import os
from joblib import load
import logging
from multiprocessing import Process
import numpy as np


model_path = os.path.abspath('/content/isolation_forest.joblib')
clf = load(model_path)
consumer = KafkaConsumer(TRANSACTIONS_TOPIC, bootstrap_servers=['127.0.0.1:9092'],
                          group_id = TRANSACTIONS_CONSUMER_GROUP, auto_offset_reset='earliest',
  enable_auto_commit=True)

producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])

def detect():


    while True:
        message = consumer.poll()
        if message is None:
            continue
        # if message.error():
        #     logging.error("Consumer error: {}".format(message.error()))
        #     continue
        print(message)
        # Message that came from producer
        record = json.loads(message.value().decode('utf-8'))
        data = record["data"]

        prediction = clf.predict(data)

        # If an anomaly comes in, send it to anomalies topic
        if prediction[0] == -1:
            score = clf.score_samples(data)
            record["score"] = np.round(score, 3).tolist()

            _id = str(record["id"])
            record = json.dumps(record).encode("utf-8")

            producer.produce(topic=ANOMALIES_TOPIC,
                             value=record,
                             key=_id)
            producer.flush()

    consumer.close()

# Call the detect function
detect()

KeyboardInterrupt: 

In [ ]:
def decode_kafka_item(item):
  print(item)

BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE=64
train_ds = tfio.IODataset.from_kafka('transactions', partition=0, offset=0)
train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
decode_kafka_item


In [ ]:

consumer = KafkaConsumer(
     TRANSACTIONS_TOPIC,
     bootstrap_servers=['127.0.0.1:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [82]:
from json import loads
consumer = KafkaConsumer(TRANSACTIONS_TOPIC, bootstrap_servers=['127.0.0.1:9092'],
                          group_id = TRANSACTIONS_CONSUMER_GROUP, auto_offset_reset='earliest',
  enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
for message in consumer:
    message = message.value
    collection.insert_one(message)
    print('{} added to {}'.format(message, collection))